In [1]:
# General imports ..
import os
import random
import logging
import urllib3
import warnings
import json
import requests

from tqdm import tqdm
from dotenv import load_dotenv

# langchain imports ..
from langchain.retrievers import WikipediaRetriever
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
import google.generativeai as genai

from urls import base_url, api_url, get_sub_url, get_sub_chap_url, add_ai_url
from logger import setup_logger

load_dotenv()
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))

from get_topics import *

# 1. Generate answers ..
def gen_content_with_openai(prompt):
    model_name = "gpt-4-1106-preview"
    temperature = 0.5
    model = OpenAI(model_name=model_name, temperature=temperature)
    output = model(prompt)
    return output

# 2.  Gmini Pro ..
def gen_content_with_gemini_pro(prompt) :
    llm = ChatGoogleGenerativeAI(model="gemini-pro", temperature=0.5)
    out = llm.invoke(prompt)
    return out.content


In [2]:
topics = get_topics_grade_wise('Mathematics', 12)
topics[3]

{'chapterName': 'Relations and functions : Types of Relations',
 'topicName': 'Symmetric',
 'topic_grade': '12 (Science)',
 'subject': 'Mathematics'}

In [11]:
def gen_html_prompt(topic, chapter, grade, subject, language):
    # Define HTML instructions
    html_instructions = {
        "heading": "<h{level}>",
        "paragraph": "<p>",
        "strong": "<strong>",
        "subscript": "<sub>",
        "superscript": "<sup>",
        "mathjax_inline": "$$",
        "mathjax_block": "$$"
    }

    # Construct the prompt with instructions, paragraph, and HTML format instructions
    prompt = (
        f"{html_instructions['paragraph']}Please embed HTML tags as necessary to format the following assignment for the topic '{topic}':\n\n"
        f"{html_instructions['heading'].format(level=2)}Chapter: {chapter}{html_instructions['heading'].format(level=2)}"
        f"{html_instructions['paragraph']}Grade: {grade}{html_instructions['paragraph']}"
        f"{html_instructions['heading'].format(level=3)}{subject} Assignment - {topic} ({language}){html_instructions['heading'].format(level=3)}"
        f"{html_instructions['paragraph']}This is your assignment content for {topic} in {language}:{html_instructions['paragraph']}"
        f"{html_instructions['paragraph']}1. Write a short essay on {topic}. For example, discuss the significance of vectors in physics and engineering applications. Alternatively, explore historical developments in the understanding of vectors.{html_instructions['paragraph']}"
        f"{html_instructions['paragraph']}2. Solve the problems from {chapter}. For instance, calculate the resultant vector of two given vectors. Alternatively, apply vector addition and subtraction to real-world scenarios, such as displacement in navigation.{html_instructions['paragraph']}"
        f"{html_instructions['paragraph']}3. Research and present a project on {topic}. For example, explore real-world examples where vector concepts are applied, such as navigation systems or computer graphics. Alternatively, investigate the history of vector analysis and its contributions to mathematics and science.{html_instructions['paragraph']}"
        f"{html_instructions['paragraph']}4. Create a poster illustrating key concepts related to {topic}. Include visual representations and explanations to aid understanding.{html_instructions['paragraph']}"
        f"{html_instructions['paragraph']}5. Design a quiz or set of practice problems on {topic}. Include a variety of question types to assess different aspects of understanding, such as multiple choice, short answer, and problem-solving.{html_instructions['paragraph']}"
        
    )

    return prompt

# Example usage:
topic = "Vectors"
chapter = "Linear Algebra"
grade = "12"
subject = "Mathematics"
language = "English"
assignment_prompt = gen_html_prompt(topic, chapter, grade, subject, language)
out = gen_content_with_gemini_pro(assignment_prompt)
print(out)

<h2>Chapter: Linear Algebra</h2>
<p>Grade: 12</p>
<h3>Mathematics Assignment - Vectors (English)</h3>
<p>This is your assignment content for Vectors in English:</p>
<ol>
  <li>Write a short essay on Vectors. For example, discuss the significance of vectors in physics and engineering applications. Alternatively, explore historical developments in the understanding of vectors.</li>
  <li>Solve the problems from Linear Algebra. For instance, calculate the resultant vector of two given vectors. Alternatively, apply vector addition and subtraction to real-world scenarios, such as displacement in navigation.</li>
  <li>Research and present a project on Vectors. For example, explore real-world examples where vector concepts are applied, such as navigation systems or computer graphics. Alternatively, investigate the history of vector analysis and its contributions to mathematics and science.</li>
  <li>Create a poster illustrating key concepts related to Vectors. Include visual representations

In [16]:
from translate import translate

print(translate(out, "Hindi"))

<h2>अध्याय: रैखिक बीजगणित</h2>
<p>ग्रेड: 12</p>
<h3>गणित असाइनमेंट - वेक्टर (अंग्रेजी)</h3>
<p>अंग्रेजी में वेक्टर के लिए यह आपकी असाइनमेंट सामग्री है:</p>
<ol>
  <li>वेक्टर पर एक लघु निबंध लिखें। उदाहरण के लिए, भौतिकी और इंजीनियरिंग अनुप्रयोगों में वैक्टर के महत्व पर चर्चा करें। वैकल्पिक रूप से, वैक्टर की समझ में ऐतिहासिक विकास का पता लगाएं।</li>
  <li>रैखिक बीजगणित से समस्याओं को हल करें। उदाहरण के लिए, दिए गए दो सदिशों के परिणामी सदिश की गणना करें। वैकल्पिक रूप से, वास्तविक दुनिया के परिदृश्यों में वेक्टर जोड़ और घटाव लागू करें, जैसे नेविगेशन में विस्थापन।</li>
  <li>वेक्टर पर शोध करें और एक प्रोजेक्ट प्रस्तुत करें। उदाहरण के लिए, वास्तविक दुनिया के उदाहरण देखें जहां वेक्टर अवधारणाएं लागू होती हैं, जैसे नेविगेशन सिस्टम या कंप्यूटर ग्राफिक्स। वैकल्पिक रूप से, वेक्टर विश्लेषण के इतिहास और गणित और विज्ञान में इसके योगदान की जांच करें।</li>
  <li>वेक्टर से संबंधित प्रमुख अवधारणाओं को दर्शाने वाला एक पोस्टर बनाएं। समझने में सहायता के लिए दृश्य प्रतिनिधित्व और स्पष्टीकरण शामिल करें।</li>
